In [1]:
import cv2
import numpy as np
import math

# Object detection


In [2]:


class ObjectDetection:
    def __init__(self, weights_path="dnn_model/yolov4.weights", cfg_path="dnn_model/yolov4.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608

        # Load Network
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.classes = []
        self.load_class_names()
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="dnn_model/classes.txt"):

        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        self.colors = np.random.uniform(0, 255, size=(80, 3))
        return self.classes

    def detect(self, frame):
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)



# object tracking

In [3]:

def compute_center(box):
    x, y, w, h = box
    center_x, center_y = int((x * 2 + w) / 2), int((y * 2 + h) / 2)
    return center_x, center_y


def compute_distance(point1, point2):
    return math.hypot(point1[0] - point2[0], point1[1] - point2[1])


def track_objects(tracked_objects, current_frame_points, track_id):
    for object_id, prev_frame_point in list(tracked_objects.items()):
        object_found = False
        for this_frame_point in current_frame_points:
            if compute_distance(prev_frame_point, this_frame_point) < 20:
                tracked_objects[object_id] = this_frame_point
                object_found = True
                current_frame_points.remove(this_frame_point)
                break
        if not object_found:
            del tracked_objects[object_id]
            
    for this_frame_point in current_frame_points:
        tracked_objects[track_id] = this_frame_point
        track_id += 1

    return tracked_objects, track_id

def is_crossing_line(point, prev_point, line_coordinates):
    point_1, point_2 = line_coordinates
    return ((prev_point[0] - point_1[0]) * (point_2[1] - point_1[1]) - (prev_point[1] - point_1[1]) * (point_2[0] - point_1[0])) * ((point[0] - point_1[0]) * (point_2[1] - point_1[1]) - (point[1] - point_1[1]) * (point_2[0] - point_1[0])) < 0



# case

In [4]:
object_detector = ObjectDetection()
video_capture = cv2.VideoCapture('los_angeles.mp4')

frame_count, track_id = 0, 0
previous_frame_points = {}
tracked_objects = {}
line_coordinates = [(100, 580), (1520, 580)]
crossed_count = 0
trajectories = {}

while True:
    frame_count += 1
    ret, frame = video_capture.read()

    if not ret:
        break

    class_ids, scores, boxes = object_detector.detect(frame)
    current_frame_points = [compute_center(box) for box in boxes]
    
    for box in boxes:
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if frame_count <= 2:
        tracked_objects, track_id = track_objects(tracked_objects, current_frame_points, track_id)
    else:
        tracked_objects, track_id = track_objects(tracked_objects, current_frame_points, track_id)

    # Check if the objects have crossed the line
    for obj_id, point in tracked_objects.items():
        prev_point = previous_frame_points.get(obj_id)
        if prev_point and is_crossing_line(point, prev_point, line_coordinates):
            crossed_count += 1
            
        if obj_id not in trajectories:
            trajectories[obj_id] = [point]
        else:
            trajectories[obj_id].append(point)    
            
        # Draw trajectory of each object
        for point_idx in range(1, len(trajectories[obj_id])):
            cv2.line(frame, trajectories[obj_id][point_idx - 1], trajectories[obj_id][point_idx], (0, 0, 255), 1)
            
        cv2.circle(frame, point, 5, (0, 0, 255), -1)
        cv2.putText(frame, str(obj_id), point, 0, 0.5, (0, 0, 255), 1)
        
    previous_frame_points = {k: v for k, v in tracked_objects.items()}

   # Draw the stationary line on the frame
    cv2.line(frame, *line_coordinates, (0, 255, 0), 2)
    cv2.putText(frame, f'Crossed count: {crossed_count}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    
    cv2.imshow('Frame', frame)

    if cv2.waitKey(1) == 27:
        break

video_capture.release()
cv2.destroyAllWindows()

Loading Object Detection
Running opencv dnn with YOLOv4
